# HAMSTER omniwheel robot – motor selection

Interactive analysis using the generated Jacobian from `jacobians`. Edit the parameters in the next cell and run the analysis and plotting cells.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from jacobians import hamster_jacobian

## Parameters

Edit these and re-run the cells below to explore different designs.

In [ ]:
# Motion / loading
v = 1.0           # speed (m/s)
a = 2.0           # acceleration (m/s²)
F = 20.0          # hand force magnitude (N)
T = 0.0           # external torque (N·m)
Omega = 0.0       # angular velocity (rad/s)
phaseshift = np.pi  # phase between force and acceleration

# Robot geometry
D = 0.15           # distance from center to each wheel (m)
r = 0.04           # wheel radius (m)
theta = 0.0        # robot orientation (rad)

# Inertia
m = 5.0            # mass (kg)
I = 0.4            # moment of inertia (kg·m²)

# Drivetrain
gear_ratio = 1.0   # motor-to-wheel ratio

# Sweep resolution
phi_step = 0.1     # step for force direction (rad)

## Run analysis

Computes motor torques (force only, acceleration only, combined) and wheel speeds over force direction φ.

In [ ]:
J = hamster_jacobian(D, r, theta)
M = np.diag([m, m, I])

phi_range = np.arange(0.0, 2.0 * np.pi + 1e-12, phi_step)

tau_F_only = []
tau_a_only = []
omega_wheel = []

for phi in phi_range:
    Fx = F * np.cos(phi)
    Fy = F * np.sin(phi)
    hand_force = -np.array([Fx, Fy, T])
    tau_F_only.append(J.T @ hand_force)

    ax = a * np.cos(phi + phaseshift)
    ay = a * np.sin(phi + phaseshift)
    robot_acc = np.array([ax, ay, 0.0])
    tau_a_only.append(J.T @ (M @ robot_acc))

    vx = v * np.cos(phi)
    vy = v * np.sin(phi)
    V = np.array([vx, vy, Omega])
    omega_wheel.append(np.linalg.solve(J, V))

tau_F_only = np.stack(tau_F_only, axis=1)
tau_a_only = np.stack(tau_a_only, axis=1)
omega_wheel = np.stack(omega_wheel, axis=1)
tau_combined = tau_F_only + tau_a_only

results = {
    "phi_range": phi_range,
    "tau_F_only": tau_F_only,
    "tau_a_only": tau_a_only,
    "tau_combined": tau_combined,
    "omega_wheel": omega_wheel,
}
print("Analysis done. phi_range length:", len(phi_range))

## Plots

Torques and wheel speeds vs force direction (degrees).

In [ ]:
%matplotlib inline

deg = np.rad2deg(phi_range)

plt.figure(1, figsize=(8, 8))
plt.clf()

plt.subplot(3, 1, 1)
plt.plot(deg, tau_F_only.T / gear_ratio)
plt.title("Motor torque – hand force only")
plt.ylabel("Torque")

plt.subplot(3, 1, 2)
plt.plot(deg, tau_a_only.T / gear_ratio)
plt.title("Motor torque – linear acceleration only")
plt.ylabel("Torque")

plt.subplot(3, 1, 3)
plt.plot(deg, tau_combined.T / gear_ratio)
plt.title("Combined (force + acceleration)")
plt.xlabel("Force direction φ (deg)")
plt.ylabel("Torque")

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(2, figsize=(8, 3))
plt.clf()
plt.plot(deg, (omega_wheel * gear_ratio).T)
plt.title("Motor speed (rad/s)")
plt.xlabel("Force direction φ (deg)")
plt.ylabel("ω")
plt.tight_layout()
plt.show()

## Inspect results (optional)

Peak torques and speeds for motor sizing.

In [ ]:
print("Max torque (force only):", np.max(np.abs(tau_F_only)) / gear_ratio)
print("Max torque (accel only):", np.max(np.abs(tau_a_only)) / gear_ratio)
print("Max torque (combined):", np.max(np.abs(tau_combined)) / gear_ratio)
print("Max wheel speed (rad/s):", np.max(np.abs(omega_wheel)) * gear_ratio)